In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pprint

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.preprocessing import LabelEncoder
from collections import Counter

import DecisionTree

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
###############################################################
#####################  书上题目5.1 #####################
###############################################################

In [62]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

datasets, labels = create_data()
train_df = pd.DataFrame(datasets, columns=labels)
predict_df = pd.DataFrame([['老年', '否', '否', '一般']], columns=labels[0:-1])

for col in range(len(labels) - 1):
    le = LabelEncoder()
    le.fit(train_df.iloc[:, col])
    train_df[labels[col]] = le.transform(train_df.iloc[:, col])
    predict_df[labels[col]] = le.transform(predict_df.iloc[:, col])

LabelEncoder()

LabelEncoder()

LabelEncoder()

LabelEncoder()

In [63]:
dt = DecisionTree.DecisionTree()
tree = dt.fit(train_df)
pred = dt.predict(predict_df)
print(pred)

['否']


In [64]:
dt = DecisionTree.DecisionTree(type = 'c4.5')
tree = dt.fit(train_df)
pred = dt.predict(predict_df)
print(pred)

['否']


In [66]:
x_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]

dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
dtc.predict(predict_df)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

array(['否'], dtype=object)

In [ ]:
###############################################################
#####################  iris数据集(鸢尾花) #####################
###############################################################

In [72]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = [
    'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
]

train_df, test_df = train_test_split(df, test_size=0.35, random_state=9)

In [74]:
x_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]
x_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]

dtc = DecisionTreeClassifier('entropy')
dtc.fit(x_train, y_train)
dtc.score(x_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

0.9811320754716981

In [73]:
dt = DecisionTree.DecisionTree()
tree = dt.fit(train_df)
print(dt.predict(test_df))
print(dt.score(test_df))

[2, 1, 2, 2, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 2, 0, 2, 1, 0, 2, 0, 2, 2, 2, 0, 1, 1, 1, 2, 0, 2, 0, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 2, 1, 0]
0.9622641509433962
